### **Spatio Temporal Window Kriging** for mapping daily Sea Level Anomalies from satellite altimetry along-track data (L3 to L4) 

This notebook can be used to produce daily Sea Level Anomalie Grids from along-track satellite altimetry data for California in 2018. The gridded data are published at SEANOE under https://doi.org/10.17882/103947.

The approach is based on Ordinary Kriging in a moving $\pm$ 5-day time window around the target day. Kriging is done by building an empirical variogram from the observations (along-track data) and fitting a sum-metric semi-variogram model, which is the base for the Kriging matrix. For details on Kriging, we recommend literature$^*$ listed below. The approach uses along-track data from CMEMS as input data (https://doi.org/10.48670/moi-00146) and produces gridded data comparable to their maps (https://doi.org/10.48670/moi-00149).

*See for details: https://github.com/mariejuhl/ST-Window-Kriging/edit/main/README.md


### 1. Import Dependencies

In [4]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import xarray as xr 
from sklearn.preprocessing import StandardScaler
import datetime
import tools
from tools import *
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

### 2. Input data, Area Selection, Mask and Subsampling

In [2]:
lat_min ,lat_max, lon_min , lon_max, extra = 32, 42, 360-128,360-116, 1

## input data 
l3 = xr.open_dataset('sample_data/L3_CALIFORNIA_2018_all_missions.nc').rename({'sla_unfiltered':'resid'})
l3 = select_data_xarray(l3,lat_min-extra, lat_max+extra, lon_min-extra, lon_max+extra, 2018, 2018).to_dataframe().dropna()

## sample grid used from CMEMS:
grid = xr.open_dataset("sample_data/DUACS_GRID_CALIFORNIA_2018.nc")
grid = select_data_xarray(grid,lat_min, lat_max, lon_min-360, lon_max-360, 2018, 2018)

# build land mask from CMEMS grid, own grid can be inlcuded: add mask for land if possible 
# otherwise prediction will be made where 'sla' is given e.g. when supplying CMEMS gridded data as grid 
grid['mask'] = np.invert(np.isnan(grid['sla']))    
grid=grid.to_dataframe().reset_index()
grid['land'] =1
grid['land'][grid['mask'].values==False]=np.nan

## reorganizing and subsampling by 1/5th
l3         = l3[['longitude','latitude','resid','source']].dropna()
l3         = l3[::5] 
l3['time'] = l3.index;
l3

,longitude,latitude,resid,source,time
time,,,,,
2018-05-28 00:50:58.257118464,243.053802,31.031515,-0.045,Jason-3,2018-05-28 00:50:58.257118464
2018-05-28 00:51:03.650018560,243.190295,31.285458,-0.014,Jason-3,2018-05-28 00:51:03.650018560
2018-05-28 10:43:35.944641024,233.856473,42.986391,0.100,Jason-3,2018-05-28 10:43:35.944641024
2018-05-28 10:43:41.337541120,234.050309,42.746587,0.004,Jason-3,2018-05-28 10:43:41.337541120
2018-05-28 10:43:46.730441216,234.242484,42.506367,0.063,Jason-3,2018-05-28 10:43:46.730441216
...,...,...,...,...,...
2018-10-10 16:10:37.362946816,238.037282,32.379925,0.201,Cryosat-2,2018-10-10 16:10:37.362946816
2018-10-10 16:10:42.079947008,238.003499,32.093354,0.152,Cryosat-2,2018-10-10 16:10:42.079947008
2018-10-10 16:10:46.796946944,237.969803,31.806769,0.130,Cryosat-2,2018-10-10 16:10:46.796946944


### 3. Producing daily Maps:

In [ ]:
# Define the time period of Kriging:
start_date = pd.to_datetime(datetime.date(2018,5,5))
end_date   = pd.to_datetime(datetime.date(2018,5,5))
run_time   = pd.date_range(start=start_date, end=end_date)

import importlib
importlib.reload(tools)


'''
Input parameters: 

l3 (DataFrame)      : DataFrame containing residual data with a time index. 'latitude', 'longitude' and 'resid' needs to be given.  
grid (DataFrame)    : Empty Grid provided containing, e.g. CMEMS grid provided with land mask to only predict ocean points
hs/ht range (tuple) : Bins in space and time: Bin number must be the same in space and time !!!     
estimator (str)     : Kind of estimation of the esxperimental variogram in space-time. Choose between 'matheron' or 'cressie'. Default is #matheron'.
func (callable)     : Function of building the space-time variogram model. 'sum-metric' tested and implememted.
plot (bool)         : Whether to plot the experimental and modeled variograms for each day.
exact (bool)        : Controlls smoothness of the prediction. True (default) sets diagonal of the Kriging matrix to zero, smoothing the result, allwing more deviation from the observations. 

Kriging (optional): 

  - if Kriging = False --> function returns 2 parameters (paras and fitting quality)
  - if Kriging = True  --> Function returns dataframe given as grid, added OK-ST and OK-ST-error

Saving Grid (optional):

    a valid path for 'save_as_csv    = /.../...' for save as CSV
    a valid path for 'save_as_netcdf = /.../...' for save as NETCDF '''


for xtime in run_time:
  df = ST_Window_Kriging( xtime                = xtime, 
                                l3             = l3,    
                                grid           = grid,
                                n_days         = 5, 
                                hs_range       = (0,200,10),
                                ht_range       = (0,5,10), 
                                estimator      = 'matheron', 
                                func           = sum_metric_model, 
                                plot           = True, 
                                Kriging        = True,
                                exact          = False, 
                                save_as_csv    = None,
                                save_as_netcdf = None)
  


  # Daily test plot for checking the results is created:
  ds = df.set_index(['time','latitude','longitude']).to_xarray()

  fig,axis = plt.subplot_mosaic("""AB
                                   AB
                                   CD""", figsize=(8,5), layout='constrained',gridspec_kw={'hspace': 0.1, 'wspace': 0.1}, )   

  a1 = axis['A'].contourf(ds.longitude,ds.latitude,ds.OK_ST[0,:,:],levels=40,cmap='RdYlBu_r', vmin=-0.1, vmax=0.3);
  plt.colorbar(a1, ax=axis['A']);axis['A'].set_title('OK-ST');
  a2 = axis['B'].contourf(ds.longitude,ds.latitude,ds.sla[0,:,:],levels = 40,cmap='RdYlBu_r', vmin=-0.1, vmax=0.3);
  plt.colorbar(a2, ax=axis['B']);axis['B'].set_title('CMEMS');

  axis['C'].hist(ds.OK_ST[0,:,:].values.flatten(), bins=100);
  axis['C'].set_title('OK-ST');
  axis['C'].set_ylim(0,45);axis['C'].set_xlim(-0.15,0.2)
  axis['D'].hist(ds.sla[0,:,:].values.flatten(), bins=100);
  axis['D'].set_title('CMEMS');
  axis['D'].set_ylim(0,45);axis['D'].set_xlim(-0.15,0.2)
  plt.show()


Processing time: 2018-05-05 00:00:00
Making temporal matrix...
Making distance matrix...
Making value matrix...
Fitting Results for time 2018-05-05 00:00:00:
  MAE: 0.1126, RMSE: 0.1598, R^2: 0.6048
2018-05-05 00:00:00
Preparing...


  0%|          | 0/1920 [00:00<?, ?it/s]